In [ ]:
import requests
import torch
import hashlib
import numpy as np
import json
import torch.nn as nn
import numpy as np
import string 
import random as r
from tqdm import tqdm
import math
import matplotlib.pyplot as plt
import warnings

In [ ]:
# !cp /content/drive/MyDrive/model_connector.ckpt /content

In [ ]:
# !cp /content/drive/MyDrive/model_input.ckpt /content

In [ ]:
# !cp /content/drive/MyDrive/model_output.ckpt /content

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
num_classes=32
num_epochs=2000
lr=0.0001
input_size=1
sequence_length=64
hidden_size=32
num_layers=2
batch_size=32

In [ ]:
#after getting file return json format block and transaction
def fetch_data(i):
    blockset='/content/drive/MyDrive/blockset.txt'
    with open(blockset, 'r') as f:
      dataset = json.loads(f.read())
    response = requests.get("https://blockchain.info/rawblock/{}".format(dataset[i]))
    data = response.json()
    return data

In [ ]:
def input_preprocessing(input):
  new_input={'ver':input.get('ver'),
             'prev_block':input.get('prev_block'),
             'mrkl_root':input.get('mrkl_root'),
             'time':input.get('time'),
             'bits':input.get('bits')}
  return new_input

In [ ]:
def output_checker(input,pred_nonce):
  new_input={'ver':input.get('ver'),
             'prev_block':input.get('prev_block'),
             'mrkl_root':input.get('mrkl_root'),
             'time':input.get('time'),
             'bits':input.get('bits'),
             'nonce':pred_nonce}
  return new_input

In [ ]:
def target_preprocessing(bits):
  #bits='0x1e03a30c'
  exponent=bits[2:4]
  coefficient=bits[4:]
  exponents=int('8',16)*(int(exponent,16)-int('3',16))
  target=int(coefficient,16)*(int('2',16))**exponents
  targets=format(target,'x')
  target=str(targets).zfill(64)
  return target

In [ ]:
#sha256 return hash output
def hash_256(string):
  new_out=json.dumps(string).encode('utf-8')
  return hashlib.sha256(new_out).hexdigest()

In [ ]:
def target_cal(hash,target):
  reward=0
  if hash.startswith(target):
    print('******************************Congrats You find Golden Nonce*******************************************************************')
    reward+=1
    return True,reward
  else:
    return False

In [ ]:
def prefix(target):
  for i in range(0,len(target)):
    if target[i]=='f':
      prefix_value=i
      break
  prefix=target[:prefix_value]
  return prefix

In [ ]:
def output_hash_generator(n):
  n=len(n)
  zeros='0'*n
  zeros=list(zeros)
  k=64-n
  chars=['A','B','C','D','E','F','0','1','2','3','4','5','6','7','8','9']
  data1=r.choices(chars,k=64-n)
  data=zeros+data1
  return data

In [ ]:
def encoding(dataset):
  lst=[]
  for i in dataset:
    data=int(i,16)
    lst.append(data)

  data=torch.tensor(lst)
  data=data.unsqueeze(0)
  data=torch.transpose(data,0,1)
  #data=data.unsqueeze(0)

  data=torch.tensor(data,dtype=torch.float)
  return data

In [ ]:
def getting_nonce(input):
  nonce=input.get('nonce')
  return nonce

In [ ]:
def wrapper(dataset):
    
    dataset=input_preprocessing(dataset)
    dataset=hash_256(dataset)
    dataset=encoding(dataset)
    
    return dataset

In [ ]:
def output_wrapper():
  out=output_checker(input,pred_nonce)
  out=hash_256(out)
  return out

In [ ]:
def converter(ins):
  lst=[]
  for i in ins:
    lst+=i
    lst=''.join(lst)
  return lst

In [ ]:
def weight_init(tensor):
  return torch.nn.init.xavier_normal_(tensor, gain=1.0)

In [ ]:
def nonce_converter(dat):
  dat=str(dat.item())
  for i in range(len(dat)):
    if dat[i]=='.':
      index=i
  counter=0
  dat=dat[index+1::]

  return dat

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self):
    path='/content/drive/MyDrive/dataset .txt'
    with open(path, 'r') as f:
      dataset = json.loads(f.read())
    self.dataset=dataset
  
  def __len__(self):
    return batch_size
  
  def __getitem__(self,index):
    global aamada
    aamada=self.dataset[index]
    dict_dataset=dict(self.dataset[index])
    bits=dict_dataset["bits"]   #take bits field from data
    nonce=dict_dataset["nonce"]
    bits=hex(bits)  #Convert bit to hex
    target_bit=target_preprocessing(bits)   #Preprocessing bit to hash
    target_bit=prefix(target_bit)   #It will find prefix
    target_hash=output_hash_generator(target_bit)  #It will generate random output from prefix data
    target_hash=converter(target_hash)   #Convert target hash to list
    target_encoded=encoding(target_hash)
    input_data=wrapper(self.dataset[index])
    return input_data,target_encoded

In [ ]:
loader=Dataset()
train_loader=torch.utils.data.DataLoader(loader,batch_size=batch_size,shuffle=True)

In [ ]:
class Connector(nn.Module):
  def __init__(self,input_size,hidden_size,num_layers,num_classes,batch_size,decider):
    super(Connector,self).__init__()
    self.num_layers=num_layers
    self.hidden_size=hidden_size
    self.input_size=input_size
    self.num_classes=num_classes
    self.decider=decider
    
    self.GRU_3=nn.GRU(input_size*64,hidden_size,num_layers*2,batch_first=True,bidirectional=True)
    self.GRU_4=nn.GRU(input_size*64,hidden_size,num_layers*2,batch_first=True,bidirectional=True)
 
    self.fc1=nn.Linear(131072,hidden_size*24,bias=True)
    self.fc2=nn.Linear(hidden_size*24,hidden_size,bias=True)
    self.fc3=nn.Linear(hidden_size,1,bias=True)

    self.activation_out=nn.ReLU()
    self.acti=nn.Sigmoid()
    self.dropout = nn.Dropout(0.5)


    # self.batch_norm=nn.BatchNorm1d(hidden_size)



  def forward(self,x):
        h3=torch.zeros(self.num_layers*4,batch_size,self.hidden_size)
        h3=weight_init(h3)
        h4=torch.zeros(self.num_layers*4,batch_size,self.hidden_size)
        h4=weight_init(h4)
        if self.decider:
          out,_=self.GRU_3(x,h3)
          out=self.acti(out)
          # out=self.batch_norm(out)
          out,_=self.GRU_4(out,h4)
          out=self.acti(out)
          # out=self.batch_norm(out)
          # out=out.reshape(out.shape[0],-1)
          out=torch.flatten(out)
          out=self.fc1(out)
          out_hash=self.fc2(out)
          # out_hash=out_hash[1]
          return out_hash
        else:
          out_nonce=self.fc3(x)
          out=torch.reshape(out_nonce[0],(1,1))
          out_nonce=nonce_converter(out)
          out_nonce=int(out_nonce)
          out_nonce=torch.tensor(out_nonce,dtype=torch.int32)
          out_nonce=abs(out_nonce)
          out_nonce=torch.tensor(out_nonce)

          return out_nonce


In [ ]:
class GRU(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers,num_classes,batch_size):
        super(GRU,self).__init__()
        self.num_layers=num_layers
        self.hidden_size=hidden_size
        self.input_size=input_size
        self.num_classes=num_classes
        
        self.GRU_0=nn.GRU(input_size,hidden_size,num_layers*2,batch_first=True)
        # self.GRU_1=nn.GRU(input_size*64,hidden_size*2,num_layers*2,batch_first=True)
        # self.GRU_2=nn.GRU(input_size*256,hidden_size*8,num_layers*2,batch_first=True)
        self.acti=nn.Sigmoid()
        self.dropout = nn.Dropout(0.5)
        # self.batch_norm=nn.BatchNorm1d(hidden_size)




    def forward(self,x):
        h0=torch.zeros(self.num_layers*2,batch_size,self.hidden_size)
        h0=weight_init(h0)
        # h1=torch.zeros(self.num_layers*2,batch_size,self.hidden_size*2)
        # h1=weight_init(h1)
        # h2=torch.zeros(self.num_layers*2,batch_size,self.hidden_size*8)
        # h2=weight_init(h2)
        
 
        #Forward Propagation
        out,_=self.GRU_0(x,h0)
        out=self.acti(out)
        # out=self.batch_norm(out)
        # out,_=self.GRU_1(out,h1)
        # out=self.acti(out)
        return out

In [ ]:
model_input=GRU(input_size, hidden_size, num_layers, num_classes,batch_size)
model_connector_hash=Connector(input_size, hidden_size, num_layers, num_classes,batch_size,decider=True)
model_connector_nonce=Connector(input_size, hidden_size, num_layers, num_classes,batch_size,decider=False)

In [ ]:
criterion=nn.MSELoss()
nonce_acti=nn.Tanh()
input_acti=nn.Sigmoid()
criterion_2=nn.L1Loss()

optimizer_hash = torch.optim.Adadelta(model_connector_hash.parameters(), lr=lr,weight_decay=0.0001) 
optimizer_nonce = torch.optim.Adadelta(model_connector_nonce.parameters(), lr=lr,weight_decay=0.0001) 

loss_n=[]
loss_h=[]
nonce_val=[]
epoch_val=[]
counter=0

for i in tqdm(range(0,num_epochs)):
  for j,(input, output) in enumerate(train_loader):
        input=input_acti(input)
        output=input_acti(output)
        score=model_input(input)  #feed input
  
        
        score_2=model_input(output)   #Feed target data input to modelone

        real_input=torch.cat([score,score_2],dim=2)  #Concatenate two different input for second model
        out_hash=model_connector_hash(real_input)    #Connector model for hash
        out_nonce=model_connector_nonce(out_hash)     #Connector model for nonce

        nonce=getting_nonce(aamada)    #Get nonce from data
        outputwithnonce=output_checker(aamada,out_nonce.item())   #It will generate dictionary data with real data and pred nonce
        outputhash=hash_256(outputwithnonce)  #It will convert pred nonce dictionary to hash
        encoded_output=encoding(outputhash)   #Hash encoding
        
        target_hash=aamada.get('next_block')
        target_hash=str(target_hash)
        target=target_cal(outputhash,target_hash)   #Just check target   

        loss_hash=criterion(out_hash,output)   #loss cal for hash 
        

        nonce=torch.tensor(nonce)
        out_nonce=torch.tensor(out_nonce)
        nonces=torch.log(nonce)
        out_nonces=torch.log(out_nonce)
        loss_nonce=criterion_2(out_nonces,nonces)   #loss for nonce

        loss_nonce=torch.tensor(loss_nonce,requires_grad=True)
        nonce_val.append(out_nonce)
        loss_h.append(loss_hash)
        loss_n.append(loss_nonce)
        epoch_val.append(i)
        
        optimizer_hash.zero_grad()
        loss_hash.backward()
        optimizer_hash.step()
        

        optimizer_nonce.zero_grad()
        loss_nonce.backward()
        optimizer_nonce.step()


      #  if counter%2==0:
        # print('Epoch[{}/{}],loss hash:{},loss_nonce:{},Expected nonce:{},Predicted nonce:{}'
        #     .format(j,len(train_loader),loss_h,loss_n,output,nonce))

  0%|          | 0/2000 [00:00<?, ?it/s]

torch.Size([32, 64, 1])


  0%|          | 1/2000 [00:04<2:23:47,  4.32s/it]

Epoch[0/1],loss hash:[tensor(0.0368, grad_fn=<MseLossBackward0>)],loss_nonce:[tensor(0.8460, requires_grad=True)],Expected nonce:tensor([[[0.5000],
         [0.5000],
         [0.5000],
         ...,
         [0.9997],
         [0.9991],
         [1.0000]],

        [[0.5000],
         [0.5000],
         [0.5000],
         ...,
         [0.9933],
         [0.9991],
         [1.0000]],

        [[0.5000],
         [0.5000],
         [0.5000],
         ...,
         [0.9975],
         [1.0000],
         [1.0000]],

        ...,

        [[0.5000],
         [0.5000],
         [0.5000],
         ...,
         [0.9820],
         [0.9999],
         [1.0000]],

        [[0.5000],
         [0.5000],
         [0.5000],
         ...,
         [0.9975],
         [0.9991],
         [0.9991]],

        [[0.5000],
         [0.5000],
         [0.5000],
         ...,
         [0.5000],
         [1.0000],
         [0.9991]]]),Predicted nonce:1639830024
torch.Size([32, 64, 1])


  0%|          | 2/2000 [00:05<1:30:38,  2.72s/it]

...

Epoch[0/1],loss hash:[tensor(0.0368, grad_fn=<MseLossBackward0>), tensor(0.0361, grad_fn=<MseLossBackward0>), tensor(0.0353, grad_fn=<MseLossBackward0>), tensor(0.0356, grad_fn=<MseLossBackward0>), tensor(0.0370, grad_fn=<MseLossBackward0>)],loss_nonce:[tensor(0.8460, requires_grad=True), tensor(0.7222, requires_grad=True), tensor(0.4617, requires_grad=True), tensor(2.0248, requires_grad=True), tensor(0.2207, requires_grad=True)],Expected nonce:tensor([[[0.5000],
         [0.5000],
         [0.5000],
         ...,
         [1.0000],
         [1.0000],
         [0.9991]],

        [[0.5000],
         [0.5000],
         [0.5000],
         ...,
         [0.9526],
         [1.0000],
         [0.9999]],

        [[0.5000],
         [0.5000],
         [0.5000],
         ...,
         [0.7311],
         [0.5000],
         [1.0000]],

        ...,

        [[0.5000],
         [0.5000],
         [0.5000],
         ...,
         [0.8808],
         [0.8808],
         [0.8808]],

        [[0.5000]

  0%|          | 6/2000 [00:12<59:09,  1.78s/it]  

Epoch[0/1],loss hash:[tensor(0.0368, grad_fn=<MseLossBackward0>), tensor(0.0361, grad_fn=<MseLossBackward0>), tensor(0.0353, grad_fn=<MseLossBackward0>), tensor(0.0356, grad_fn=<MseLossBackward0>), tensor(0.0370, grad_fn=<MseLossBackward0>), tensor(0.0362, grad_fn=<MseLossBackward0>)],loss_nonce:[tensor(0.8460, requires_grad=True), tensor(0.7222, requires_grad=True), tensor(0.4617, requires_grad=True), tensor(2.0248, requires_grad=True), tensor(0.2207, requires_grad=True), tensor(1.2020, requires_grad=True)],Expected nonce:tensor([[[0.5000],
         [0.5000],
         [0.5000],
         ...,
         [0.9997],
         [0.9933],
         [0.9820]],

        [[0.5000],
         [0.5000],
         [0.5000],
         ...,
         [0.9997],
         [0.8808],
         [0.9820]],

        [[0.5000],
         [0.5000],
         [0.5000],
         ...,
         [1.0000],
         [0.9933],
         [1.0000]],

        ...,

        [[0.5000],
         [0.5000],
         [0.5000],
         .

  0%|          | 6/2000 [00:13<1:14:20,  2.24s/it]


KeyboardInterrupt: ignored

In [ ]:
# connector = torch.load('model_connector.ckpt')
# model_connector.load_state_dict(connector)

In [ ]:
# modelinput = torch.load('model_input.ckpt')
# model_input.load_state_dict(modelinput)

In [ ]:
# modeloutput = torch.load('model_output.ckpt')
# model_output.load_state_dict(modeloutput)

In [ ]:
# torch.save(model_input.state_dict(),'model_input.ckpt')
# torch.save(model_connector_hash.state_dict(),'model_connector_hash.ckpt')
# torch.save(model_connector_nonce.state_dict(),'model_connector_nonce.ckpt')

In [ ]:
# from google.colab import files
# files.download('model_input.ckpt')

In [ ]:
# from google.colab import files
# files.download('model_connector_hash.ckpt')

In [ ]:
from google.colab import files
files.download('model_connector_nonce.ckpt')